In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from patsy import dmatrices
pd.options.mode.chained_assignment = None

from sklearn import preprocessing

from sklearn.metrics import accuracy_score, roc_curve, auc, classification_report, confusion_matrix

from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.learning_curve import learning_curve
from sklearn.grid_search import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from __future__ import division
import pprint

import dateutil
%matplotlib inline

In [4]:
df = pd.read_csv("~/donorschoose/opendata_projects.csv")

In [5]:
df.head()

,_projectid,_teacher_acctid,_schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,...,students_reached,total_donations,num_donors,eligible_double_your_impact_match,eligible_almost_home_match,funding_status,date_posted,date_completed,date_thank_you_packet_mailed,date_expiration
0,7342bd01a2a7725ce033a179d22e382d,5c43ef5eac0f5857c266baa1ccfa3d3f,9e72d6f2f1e9367b578b6479aa5852b7,3.600097e+11,40.688454,-73.910432,Brooklyn,NY,11207,urban,...,0,251,1,f,f,completed,2002-09-13 00:00:00,2002-09-23 00:00:00,2003-01-27 00:00:00,2003-12-31 00:00:00
1,b56b502d25666e29550d107bf7e17910,57426949b47700ccf62098e1e9b0220c,4a06a328dd87bd29892d73310052f45f,3.600077e+11,40.770233,-73.950760,New York,NY,10075,NaN,...,0,125,1,f,f,completed,2002-09-16 00:00:00,2002-09-19 00:00:00,2002-12-19 00:00:00,2003-12-31 00:00:00
2,3a88a47f97bd0c9b7c6a745bcd831ce3,7a20087fac86232883537cab45c0a6c9,66d08b506d2f3c30dec9e6fdb03cc279,3.600122e+11,40.729763,-73.816925,Flushing,NY,11367,urban,...,0,2110,1,f,f,completed,2002-09-17 00:00:00,2003-03-17 00:00:00,2004-06-14 00:00:00,2003-12-31 00:00:00
3,0b8a4f8eb1822ab6dd5cd9ffb918ece3,e5d28482308e2007992ae8b358b31bce,9bc7d4d92193a453669b6e2a56128e87,3.600077e+11,40.701778,-74.011840,New York,NY,10004,urban,...,0,1241,1,f,f,completed,2002-09-17 00:00:00,2002-09-17 00:00:00,2003-12-18 00:00:00,2003-12-31 00:00:00
4,45235cb116b3b35cd86d1c3a63961d7d,214d75bd0e6ce3e7085278ddd8f26daa,f7f12dee0009e6f9afa301bcca9d5070,NaN,40.815018,-73.856094,Bronx,NY,10473,urban,...,0,83,1,f,f,completed,2002-09-17 00:00:00,2002-09-17 00:00:00,2004-01-28 00:00:00,2003-12-31 00:00:00


In [26]:
#Create holdout set with projects that are still running
holdout = df[df['funding_status'] == "live"]
len(holdout)

24559

In [27]:
#Throw out reallocated and hold out set
df2 = df[df['funding_status'] != "reallocated"]
df2 = df2[df2['funding_status'] != "live"]
per_remaining = len(df2)/len(df)
print "Percent remaining: %0.2f" %(per_remaining*100)

Percent remaining: 96.28


In [28]:
df2['RESP'] = 0
df2['RESP'][df2['funding_status'] == 'completed'] = 1

In [29]:
features = df2.columns.values.tolist()
print features

['_projectid', '_teacher_acctid', '_schoolid', 'school_ncesid', 'school_latitude', 'school_longitude', 'school_city', 'school_state', 'school_zip', 'school_metro', 'school_district', 'school_county', 'school_charter', 'school_magnet', 'school_year_round', 'school_nlns', 'school_kipp', 'school_charter_ready_promise', 'teacher_prefix', 'teacher_teach_for_america', 'teacher_ny_teaching_fellow', 'primary_focus_subject', 'primary_focus_area', 'secondary_focus_subject', 'secondary_focus_area', 'resource_type', 'poverty_level', 'grade_level', 'vendor_shipping_charges', 'sales_tax', 'payment_processing_charges', 'fulfillment_labor_materials', 'total_price_excluding_optional_support', 'total_price_including_optional_support', 'students_reached', 'total_donations', 'num_donors', 'eligible_double_your_impact_match', 'eligible_almost_home_match', 'funding_status', 'date_posted', 'date_completed', 'date_thank_you_packet_mailed', 'date_expiration', 'RESP']


In [30]:
for feature in features:
    per = (len(df2[feature]) - df2[feature].count())/len(df2[feature])
    if per > 0.0:
        print feature, "has %0.2f percent missing values" % (per*100)

school_ncesid has 5.57 percent missing values
school_zip has 0.00 percent missing values
school_metro has 11.23 percent missing values
school_district has 0.04 percent missing values
school_county has 0.00 percent missing values
teacher_prefix has 0.00 percent missing values
primary_focus_subject has 0.00 percent missing values
primary_focus_area has 0.00 percent missing values
secondary_focus_subject has 30.71 percent missing values
secondary_focus_area has 30.71 percent missing values
resource_type has 0.01 percent missing values
grade_level has 0.00 percent missing values
vendor_shipping_charges has 4.11 percent missing values
sales_tax has 4.11 percent missing values
payment_processing_charges has 4.11 percent missing values
fulfillment_labor_materials has 4.11 percent missing values
students_reached has 0.02 percent missing values
date_completed has 29.95 percent missing values
date_thank_you_packet_mailed has 35.63 percent missing values
date_expiration has 0.02 percent missing v

In [65]:
#TO DO
#general:    
    #decide if i should only use recent projects
    #for each column, check missing or 0 values, and decide how to handle
    #fill in urban by creating list of zips with missing and looking up
    #create script to clean data
    #pickle final data
    #create script to run modeling 
#Check distros and decide if valuable:
    #students reached = how many 0's, than how many nans?
    #school types

In [32]:
df2['date_posted'] = df2['date_posted'].map(lambda x: pd.to_datetime(x))
df2['date_expiration'] = df2['date_expiration'].map(lambda x: pd.to_datetime(x))
df2['date_completed'] = df2['date_completed'].map(lambda x: pd.to_datetime(x))
df2['date_thank_you_packet_mailed'] = df2['date_thank_you_packet_mailed'].map(lambda x: pd.to_datetime(x))

In [33]:
df2['time_to_expire'] = df2.date_expiration - df2.date_posted

In [55]:
df2['amount_optional_support'] = df2.total_price_including_optional_support - df2.total_price_excluding_optional_support 
df2['optional_support'] = 0
df2['optional_support'][df2['amount_optional_support'] > 0] = 1
df2['per_optional_support'] = df2.amount_optional_support/df2.total_price_excluding_optional_support

In [66]:
df2['month'] = df2.date_posted.map(lambda x: x.month)
df2['quarter'] = df2.date_posted.map(lambda x: x.quarter)

In [100]:
df2 = df2.sort("date_posted")

In [105]:
df2['teacher_previous_success'] = df2.groupby('_teacher_acctid')['RESP'].cumsum()
df2['school_previous_success'] = df2.groupby('_schoolid')['RESP'].cumsum()

In [109]:
df2['teacher_previous_projects'] = df2.groupby('_teacher_acctid').cumcount()+1
df2['school_previous_projects'] = df2.groupby('_schoolid').cumcount()+1

In [110]:
df2['teacher_per_success'] = df2['teacher_previous_success']/df2['teacher_previous_projects']
df2['school_per_success'] = df2['school_previous_success']/df2['school_previous_projects']

In [111]:
df2.head()

,_projectid,_teacher_acctid,_schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,...,optional_support,per_optional_support,month,quarter,teacher_previous_success,school_previous_success,teacher_previous_projects,school_previous_projects,teacher_per_success,school_per_success
0,7342bd01a2a7725ce033a179d22e382d,5c43ef5eac0f5857c266baa1ccfa3d3f,9e72d6f2f1e9367b578b6479aa5852b7,3.600097e+11,40.688454,-73.910432,Brooklyn,NY,11207,urban,...,1,0.219520,9,3,1,1,1,1,1,1
1,b56b502d25666e29550d107bf7e17910,57426949b47700ccf62098e1e9b0220c,4a06a328dd87bd29892d73310052f45f,3.600077e+11,40.770233,-73.950760,New York,NY,10075,NaN,...,1,0.219520,9,3,1,1,1,1,1,1
30,7844d34a23f05c7836c1ea64355cd851,982142b095e3df0ddebc12384e464b49,cabcc3ed826056c3969a8a99c5aeeaf6,3.600085e+11,40.829224,-73.856899,Bronx,NY,10472,urban,...,1,0.219514,9,3,1,1,1,1,1,1
29,d25a7dd645bdfb4d702f177cef5e0d8e,0c2d4535f03991165dc72092840d672b,cabcc3ed826056c3969a8a99c5aeeaf6,3.600085e+11,40.829224,-73.856899,Bronx,NY,10472,urban,...,1,0.219516,9,3,1,2,1,2,1,1
28,77e3b131ee90f011f937110e8e8e9cf4,b218f64064e174657336d229e2231962,52835c082f36edc0c90aff0c11a2e9b3,3.600079e+11,40.792306,-73.952037,New York,NY,10029,urban,...,1,0.219512,9,3,1,1,1,1,1,1


In [117]:
df2.columns.values.tolist()

['_projectid',
 '_teacher_acctid',
 '_schoolid',
 'school_ncesid',
 'school_latitude',
 'school_longitude',
 'school_city',
 'school_state',
 'school_zip',
 'school_metro',
 'school_district',
 'school_county',
 'school_charter',
 'school_magnet',
 'school_year_round',
 'school_nlns',
 'school_kipp',
 'school_charter_ready_promise',
 'teacher_prefix',
 'teacher_teach_for_america',
 'teacher_ny_teaching_fellow',
 'primary_focus_subject',
 'primary_focus_area',
 'secondary_focus_subject',
 'secondary_focus_area',
 'resource_type',
 'poverty_level',
 'grade_level',
 'vendor_shipping_charges',
 'sales_tax',
 'payment_processing_charges',
 'fulfillment_labor_materials',
 'total_price_excluding_optional_support',
 'total_price_including_optional_support',
 'students_reached',
 'total_donations',
 'num_donors',
 'eligible_double_your_impact_match',
 'eligible_almost_home_match',
 'funding_status',
 'date_posted',
 'date_completed',
 'date_thank_you_packet_mailed',
 'date_expiration',
 'RESP',

In [ ]:
'school_state+school_charter+school_magnet+ school_year_round+ school_nlns+ school_kipp+ school_charter_ready_promise+ teacher_prefix+teacher_teach_for_america+teacher_ny_teaching_fellow+primary_focus_subject+primary_focus_area+resource_type+poverty_level+grade_level+total_price_including_optional_support+students_reached+optional_support+quarter+teacher_per_success+school_per_success'

In [118]:
y, X = dmatrices('RESP ~ school_state+school_charter+school_magnet+ school_year_round+ school_nlns+ school_kipp+ school_charter_ready_promise+ teacher_prefix+teacher_teach_for_america+teacher_ny_teaching_fellow+primary_focus_subject+primary_focus_area+resource_type+poverty_level+grade_level+total_price_including_optional_support+students_reached+optional_support+quarter+teacher_per_success+school_per_success', data=df2, return_type='dataframe')

y = np.ravel(y)

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=7)

std_scale = preprocessing.StandardScaler().fit(x_train)
x_train = std_scale.transform(x_train)
x_test = std_scale.transform(x_test)

In [119]:
def always_complete(x):
    return [1] * len(x)
y_pred = always_complete(X)

print "Baseline = %0.2f" % accuracy_score(y, y_pred) 

Baseline = 0.70


In [ ]:
names, accs = [], []
for algorithm in (LogisticRegression, 
                 KNeighborsClassifier, 
                 GaussianNB,
                 SVC,
                 DecisionTreeClassifier, 
                 RandomForestClassifier):
    accuracy = np.mean(cross_val_score(algorithm(), X, y, cv = 10))
    print '%-30s %0.4f' % (algorithm.__name__, accuracy)
    names.append(algorithm.__name__)
    accs.append(accuracy)

LogisticRegression             0.8818
KNeighborsClassifier           0.7208
GaussianNB                     0.5394